## 7.4 GalleryApp

W tej aplikacji utworzymy prostą aplikację pełniącą rolę galerii zdjęć wykonanych aparatem. Będzie ona zawierać upoważnienia, bazę danych `SQLite`, dane będziemy wyświetlać w `RecyclerView`, obsługa aparatu odbędzie się za pomocą `Implicit Intent` - czyli otworzymy aplikację obsługującą aparat, wykonamy zdjęcie i prześlemy je do naszej aplikacji, następnie dodamy je do bazy danych i wyświetlimy w `RecyclerView`. Aplikacja będzie zawierać `Jetpack Navigation` oraz `Bottom Navigation`.

### **Bottom Navigation**

Rozpocznijmy od dodania dwóch fragmentów oraz nawigacji. Dodajmy dwa puste fragmenty `GalleryFragment` oraz `AddPictureFragment` oraz `navigation`

```xml
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/galleryFragment">

    <fragment
        android:id="@+id/addPictureFragment"
        android:name="pl.udu.uwr.pum.galleryappjava.fragments.AddPictureFragment"
        android:label="fragment_add_picture"
        tools:layout="@layout/fragment_add_picture" />
    <fragment
        android:id="@+id/galleryFragment"
        android:name="pl.udu.uwr.pum.galleryappjava.fragments.GalleryFragment"
        android:label="fragment_gallery"
        tools:layout="@layout/fragment_gallery" />
</navigation>
```

`GalleryFragment` ustawiamy jako domowy. Następnie dodajmy `menu` dla `Bottom Navigation`

```xml
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/galleryFragment"
        android:icon="@drawable/ic_gallery"
        android:title="Gallery" />
    <item
        android:id="@id/addPictureFragment"
        android:icon="@drawable/ic_add"
        android:title="Add" />
</menu>
```

Dodajmy `FragmentContainer` i `BottomNavigationView` do layoutu głównej aktywności

```xml
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/nav_host_fragment"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        app:defaultNavHost="true"
        app:navGraph="@navigation/navigation"/>

    <com.google.android.material.bottomnavigation.BottomNavigationView
        android:id="@+id/bottom_nav_view"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:menu="@menu/bottom_menu" />

</LinearLayout>
```

Przejdźmy do `MainActivity` i połączmy nawigację

In [ ]:
class MainActivity : AppCompatActivity() {

    private lateinit var binding: ActivityMainBinding

    private val navController: NavController by lazy {
        val navHostFragment = supportFragmentManager.findFragmentById(R.id.nav_host_fragment)
                as NavHostFragment
        navHostFragment.findNavController()
    }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        binding = ActivityMainBinding.inflate(layoutInflater)
        setContentView(binding.root)

        binding.bottomNavView.setupWithNavController(navController)
    }
}

### **Permission**

Dodajmy upoważnienie na wykorzystanie aparatu do aplikacji. Przejdźmy do `AndroidManifest.xml` i dodajmy odpowiedni wpis

```xml
<uses-permission android:name="android.permission.CAMERA" />
```

W layoucie `AddPictureFragment` dodamy dwa przyciski odpowiedzialne za wykonanie zdjęcia (sprawdzenie upoważnienia oraz otworzenie aplikacji obsługującej aparat) oraz za zapis do bazy danych. Mamy również `EditText` w który będziemy wpisywać tytuł zdjęcia oraz `ImageView` do którego będziemy przekazywać wykonane zdjęcie.

```xml
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <EditText
        android:id="@+id/edit_text_title"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="20dp"
        android:textSize="24sp"
        android:hint="Title"
        android:inputType="textCapWords"
        android:autofillHints="title" />

    <LinearLayout
        android:layout_width="wrap_content"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:gravity="center"
        android:layout_gravity="center"
        android:orientation="vertical">

        <ImageView
            android:id="@+id/image_view_picture"
            android:layout_width="300dp"
            android:layout_height="300dp"
            android:layout_margin="40dp"
            android:contentDescription="picture" />

    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

    <Button
        android:id="@+id/button_camera"
        android:layout_width="0dp"
        android:layout_weight="1"
        android:layout_margin="25dp"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:text="otwórz aparat"
        android:gravity="center"/>

    <Button
        android:id="@+id/button_save_picture"
        android:layout_width="0dp"
        android:layout_weight="1"
        android:layout_height="wrap_content"
        android:layout_margin="25dp"
        android:gravity="center"
        android:layout_gravity="center"
        android:text="zapisz"/>
    </LinearLayout>
</LinearLayout>
```

Przy naciśnięciu przycisku **zapisz** wpierw sprawdzimy czy aplikacja posiada uprawnienie, następnie wyślemy odpowiedni `Intent`. W pierwszym kroku dodajmy `ViewBinding`

In [ ]:
class AddPictureFragment : Fragment(){

    private lateinit var binding: FragmentAddPictureBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentAddPictureBinding.inflate(inflater, container, false)
        return binding.root
    }

Zacznijmy od dwóch `ActivityResultLauncher`
- `requestCameraPermissionLauncher` - implementuje metodę `RequestPermission`, wykorzystamy do otworzenia dialogu z możliwością udzielenia upoważnienia
- `resultLauncherCamera` - implementacja metody `StartActivityForResult`, wykorzystamy przy wysłaniu intentu z żądaniem wykonania zdjęcia - dzięki tej metodzie możemy obsłużyć wartość zwrotną.

In [ ]:
private val requestCameraPermissionLauncher = registerForActivityResult(
    ActivityResultContracts.RequestPermission()
) {
    if (it){ launchCamera() }
}

Jeżeli posiadamy odpowiednie upoważnienie wykonujemy metodę `launchCamera`, którą zaimplementujemy nieco później.

Teraz przejdźmy do `resultLauncherCamera`

In [ ]:
private val resultLauncherCamera = registerForActivityResult(
    ActivityResultContracts.StartActivityForResult()) { result ->

Jeżeli zdjęcie zostanie wykonane prawidłowo i dostaniemy dane zwrotne (sprawdzamy w warunku), odbieramy dane za pomocą `Intent`

In [ ]:
if (result.resultCode == RESULT_OK) {
    val data: Intent? = result.data

Następnie musimy je rozpakować, dane w tym przypadku otrzymujemy w postaci `Bitmap`

In [ ]:
if (result.resultCode == RESULT_OK) {
    val data: Intent? = result.data
    val imageBitmap = data?.extras?.get("data") as Bitmap
    binding.imageViewPicture.setImageBitmap(imageBitmap)
}

Klucz "data", wykorzystany w metodzie `get` jest standardową nazwą nadaną automatycznie. Następnie ustawiamy otrzymaną bitmapę na `ImageView` za pomocą metody `setImageBitmap`. 

Dodajmy metodę `openCamera`, którą wywołamy jako `onClick` przycisku wykonującego zdjęcie. Mamy trzy możliwości
- upoważnienie zostało nadane - wywołujemy metodę `launchCamera`
- upoważnienie zostało odrzucone - pokazujemy `Rationale`
- aplikacja jest świeżo zainstalowana i włączona pierwszy raz - pokazujemy dialog z możliwością nadania upoważnienia

Te trzy opcje chcemy obsłużyć

In [ ]:
    private fun openCamera(){
        when {ContextCompat.checkSelfPermission(
            requireContext(), Manifest.permission.CAMERA) ==
                PackageManager.PERMISSION_GRANTED -> {
            launchCamera() // włączam aplikację przez implicit intent
        }
            ActivityCompat.shouldShowRequestPermissionRationale(
                requireActivity(),
                Manifest.permission.CAMERA) -> {
                showMessageOKCancel(getString(R.string.rationale_camera)) // Rationale
            }
            else -> {
                requestCameraPermissionLauncher
                    .launch(Manifest.permission.CAMERA) // jeżeli nie to nic nie robię
            }
        }
    }

Dodajmy implementację metody `showMessageOkCancel`

In [ ]:
    private fun showMessageOKCancel(message: String) {
        AlertDialog.Builder(requireContext())
            .setMessage(message)
            .setPositiveButton("OK") { dialogInterface: DialogInterface, _: Int ->
                // jeżeli ok proszę o upoważnienie
                requestCameraPermissionLauncher.launch(Manifest.permission.CAMERA) 
                dialogInterface.dismiss()
            }
            .setNegativeButton("Cancel", null) // jeżeli nie to nic nie robię
            .create()
            .show()
    }

Pozostaje metoda `launchCamera` wysyłająca intent z prośbą o wykonanie zdjęcia

In [ ]:
private fun launchCamera(){
    val intent = Intent(MediaStore.ACTION_IMAGE_CAPTURE)
    resultLauncherCamera.launch(intent)
}

Dodajmy obsługę przycisku `buttonCamera` w metodzie `onViewCreated`

In [ ]:
binding.buttonCamera.setOnClickListener {
    openCamera()
}

### **Zapis do bazy**

W następnym kroku zapiszemy zdjęcie w bazie danych, rozpocznijmy od określenia modelu danych

In [ ]:
data class PictureModel(val title: String, val image: String) {
    var id: Int = 0

    constructor(id: Int, title: String, image: String) : this(title, image) {
        this.id = id
    }
}

Będziemy przechowywać nazwę zdjęcia i jego ścieżkę dostępu jako `String`. Przejdźmy do klasy `DBHandler`. Dodajmy podstawowe metody i pola

In [ ]:
class DBHandler(context: Context) :
    SQLiteOpenHelper(context, DATABASE_NAME, null, DATABASE_VERSION) {

    companion object{
        private const val DATABASE_VERSION = 1
        private const val DATABASE_NAME = "galleryDBKotlin"
        private const val TABLE_GALLERY = "GalleryTable"

        private const val KEY_ID = "_id"
        private const val KEY_TITLE = "title"
        private const val KEY_IMAGE = "image"
    }

    override fun onCreate(db: SQLiteDatabase?) {
        val CREATE_GALLERY_TABLE =
            "CREATE TABLE $TABLE_GALLERY(" +
                    "$KEY_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL," +
                    "$KEY_TITLE TEXT," +
                    "$KEY_IMAGE TEXT)"

        db?.execSQL(CREATE_GALLERY_TABLE)
    }

    override fun onUpgrade(db: SQLiteDatabase?, p1: Int, p2: Int) {
        db!!.execSQL("DROP TABLE IF EXISTS $TABLE_GALLERY")
        onCreate(db)
    }
}

Następnie dodajmy metodę dodającą wpis do bazy

In [ ]:
fun addToGallery(singleItem: PictureModel): Long{
    val db = this.writableDatabase

    val contentValues = ContentValues()
    contentValues.put(KEY_TITLE, singleItem.title)
    contentValues.put(KEY_IMAGE, singleItem.image)

    val result = db.insert(TABLE_GALLERY, null, contentValues)
    db.close()
    return result
}

Zmienna `result` posłuży do określenia poprawności wykonania operacji.

Drugą metodą będzie `getAllItems` zwracająca listę wszystkich elementów, lub pustą listę

In [ ]:
fun getAllItems(): List<PictureModel>{
    val itemList: MutableList<PictureModel> = mutableListOf()

    val selectQuery = "SELECT * FROM $TABLE_GALLERY"

    val db = this.readableDatabase

    try{
        val cursor: Cursor = db.rawQuery(selectQuery, null)
        if(cursor.moveToFirst()){
            do{
                val place = PictureModel(
                    cursor.getInt(0),
                    cursor.getString(1),
                    cursor.getString(2)
                )
                itemList.add(place)
            } while (cursor.moveToNext())
        }
        cursor.close()
    } catch (e: SQLiteException){
        e.printStackTrace()
        return emptyList()
    }

    return itemList
}

Wróćmy do klasy `AddPictureFragment` - chcemy zapisać ścieżkę dostępu do pliku, więc zdefiniujmy zmienną `Uri`

In [ ]:
private lateinit var pictureAbsolutePath: Uri

W obsłudze `buttonSavePicture` wpierw obsłużmy błędy, w tym celu zdefiniujmy metodę `checkForErrors`

In [ ]:
private fun checkForErrors(): Boolean{
    if (binding.editTextTitle.text.isEmpty())
        return true
    if (!this::pictureAbsolutePath.isInitialized)
        return true
    return false
}

Wyświetlimy `Toast` jeżeli pojawią się błędy

In [ ]:
binding.buttonSavePicture.setOnClickListener {
    if (checkForErrors())
        Toast.makeText(
            context, 
            getString(R.string.error_imageView), 
            Toast.LENGTH_LONG
        ).show()

W przeciwnym razie tworzymy nowy wpis na podstawie zdefiniowanego modelu

In [ ]:
    else{
        val item = PictureModel(
            binding.editTextTitle.text.toString(),
            pictureAbsolutePath.toString()
        )

i dodajemy do bazy

In [ ]:
        val dbHandler = DBHandler(requireContext())
        val addItemResult = dbHandler.addToGallery(item)

Na koniec wyświetlamy informację o powodzeniu operacji

In [ ]:
        if(addItemResult > 0)
            Toast.makeText(context, "SUCCESS", Toast.LENGTH_SHORT).show()
    }
}

Musimy jeszcze zapisać plik lokalnie na dysku (w bazie przechowujemy tylko ścieżkę dodstępu), zdefiniujmy metodę `saveImage`

In [ ]:
private fun saveImage(bitmap: Bitmap): Uri {

Utwórzmy plik i posłużmy się metodą `getDir` klasy `Context` - jeżeli katalog o zadanej nazwie nie istnieje zostaje on utworzony, jeżeli istnieje nowy plik zostanie do niego dodany. Jest to katalog w którym aplikacja posiada uprawnienia do zapisu i odczytu własnych danych.

In [ ]:
var file = requireContext().getDir("myGalleryKotlin", Context.MODE_PRIVATE)

Następnie tworzymy plik - tutaj nazwą pliku będzie uniwersalny unikalny identyfikator `UUID` wygenerowany losowo

In [ ]:
file = File(file, "${UUID.randomUUID()}.jpg")

Oraz zapisujemy otrzymaną bitmapę

In [ ]:
try {
    val stream: OutputStream = FileOutputStream(file)
    bitmap.compress(Bitmap.CompressFormat.JPEG, 100, stream)
    stream.flush()
    stream.close()
} catch (e: IOException) {
    e.printStackTrace()
}

Metoda zwraca ścieżkę absolutną jako `Uri`

In [ ]:
return Uri.parse(file.absolutePath)

Metodę `saveImage` wykonujemy przy odebraniu danych - `resultLauncherCamera`

In [ ]:
private val resultLauncherCamera = registerForActivityResult(
    ActivityResultContracts.StartActivityForResult()) { result ->
    if (result.resultCode == RESULT_OK) {
        val data: Intent? = result.data
        val imageBitmap = data?.extras?.get("data") as Bitmap
        binding.imageViewPicture.setImageBitmap(imageBitmap)
        pictureAbsolutePath = saveImage(imageBitmap) // zapis pliku oraz ścieżki
    }
}

### **RecyclerView**

W `GalleryFragment` będzie znajdował się `RecyclerView`, więc rozpocznijmy od adaptera

In [ ]:
class GalleryAdapter(private val pictures: List<PictureModel>) 
    : RecyclerView.Adapter<GalleryAdapter.ViewHolder>() {

    class ViewHolder(private val itemBinding: ItemViewBinding) 
        : RecyclerView.ViewHolder(itemBinding.root) {
        fun bind (item: PictureModel){
            itemBinding.textViewTitle.text = item.title
            itemBinding.rcImageView.setImageURI(Uri.parse(item.image))
        }
    }

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): ViewHolder {
        return ViewHolder(
            ItemViewBinding.inflate(
                LayoutInflater.from(parent.context), parent, false))
    }

    override fun onBindViewHolder(holder: ViewHolder, position: Int) {
        val item = pictures[position]
        holder.bind(item)
    }

    override fun getItemCount(): Int = pictures.size
}

dodajmy `RecyclerView` do layoutu fragmentu

```xml
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".fragments.GalleryFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recycler"
        android:layout_width="match_parent"
        android:layout_height="match_parent" />

</FrameLayout>
```

Ostatnim krokiem będzie dodanie `RecyclerView` do `GalleryFragment`

In [ ]:
class GalleryFragment : Fragment() {

    private lateinit var binding: FragmentGalleryBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentGalleryBinding.inflate(inflater, container, false)
        return binding.root
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)

        val dbHandler = DBHandler(requireContext())

        binding.recycler.apply {
            layoutManager = LinearLayoutManager(requireContext())
            adapter = GalleryAdapter(dbHandler.getAllItems())
        }
    }
}

Możemy przetestować aplikację